## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [152]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 9 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [153]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [154]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [155]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

No esta listo aun


Le echamos un vistazo para asegurar que ha ido todo bien

In [133]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,209520,6800,216320,139718,0.645886,2021-01-14
1,Aragón,35415,1200,36615,23619,0.645063,2021-01-14
2,Asturias,35420,1200,36620,31247,0.853277,2021-01-15
3,Baleares,12840,600,13440,9366,0.696875,2021-01-14
4,Canarias,41310,1300,42610,30732,0.721239,2021-01-15
5,Cantabria,17700,500,18200,14639,0.804341,2021-01-15
6,Castilla y Leon,76740,2500,79240,59411,0.749760,2021-01-14
7,Castilla La Mancha,53100,1700,54800,33809,0.616953,2021-01-15
8,Cataluña,180020,5800,185820,122577,0.659655,2021-01-14
9,C. Valenciana,91450,2900,94350,86927,0.921325,2021-01-15


..hacemos una copia para hacerle las transformaciones necesarias:

In [134]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [135]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [136]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [137]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [138]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [139]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [140]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [141]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [142]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Fecha de la ultima vacuna registrada,source_name,source
0,15/1/21,Andalucía,209520,6800,216320,139718,0.645886,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,15/1/21,Aragón,35415,1200,36615,23619,0.645063,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,15/1/21,Asturias,35420,1200,36620,31247,0.853277,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,15/1/21,Baleares,12840,600,13440,9366,0.696875,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,15/1/21,Canarias,41310,1300,42610,30732,0.721239,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,15/1/21,Cantabria,17700,500,18200,14639,0.804341,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,15/1/21,Castilla y Leon,76740,2500,79240,59411,0.749760,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,15/1/21,Castilla La Mancha,53100,1700,54800,33809,0.616953,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,15/1/21,Cataluña,180020,5800,185820,122577,0.659655,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,15/1/21,C. Valenciana,91450,2900,94350,86927,0.921325,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [143]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [144]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [145]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...
155,14/1/21,Navarra,17700.0,500.0,18200,8314,0.456813,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
156,14/1/21,País Vasco,47205.0,1400.0,48605,20003,0.411542,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
157,14/1/21,Ceuta,1005.0,200.0,1205,911,0.756017,NaN,13/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
158,14/1/21,Melilla,1005.0,200.0,1205,1098,0.911203,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [146]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [147]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [148]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Última fecha de actualización de datos','Fecha de la ultima vacuna registrada',
                                     'source_name', 'source']]

In [149]:
datos_acumulados[-10:]

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
170,15/1/21,Extremadura,32450.0,1000.0,33450,22322,0.667324,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
171,15/1/21,Galicia,56080.0,1800.0,57880,49768,0.859848,NaN,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
172,15/1/21,La Rioja,8840.0,200.0,9040,5953,0.658518,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
173,15/1/21,Madrid,147540.0,4700.0,152240,77921,0.511830,NaN,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
174,15/1/21,Murcia,38360.0,1200.0,39560,27001,0.682533,NaN,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
175,15/1/21,Navarra,17700.0,500.0,18200,9305,0.511264,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
176,15/1/21,País Vasco,47205.0,1400.0,48605,22464,0.462175,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
177,15/1/21,Ceuta,1005.0,200.0,1205,941,0.780913,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
178,15/1/21,Melilla,1005.0,200.0,1205,1230,1.020747,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
179,15/1/21,Totales,1103700.0,35700.0,1139400,768950,0.674873,NaN,NaT,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [150]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [151]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)